# Dump salaries from schedule J in Portland, OR (continuing from part 1)

In [1]:
import unicodecsv as csv
from irsx.xmlrunner import XMLRunner
import pandas as pd

In [9]:
# read the whole file of orgs with efilings from part 1 here, it's not very long
file_rows = [] 
# We're using the output of part 1
with open('orefilers.csv', 'rb') as infile:
    reader = csv.DictReader(infile)
    for row in reader:
        file_rows.append(row)
        

In [10]:
# the name of the output file
outfilename ="employees_simple.csv"
outfile = open(outfilename , 'wb')

# the header rows as they'll appear in the output
headers = ["period", "ein", "object_id", "taxpayer_name", "name", "business_name1", "business_name2", "title", "org_comp", "related_comp"]
# start up a dictwriter, ignore extra rows
dw = csv.DictWriter(outfile, headers, extrasaction='ignore')
dw.writeheader()

In [11]:
# get an XMLRunner -- this is what actually does the parsing
xml_runner = XMLRunner()

## Figure out what to extract

Data from each repeating group should go to it's own file, otherwise it won't make sense.

To figure out what to capture, I started by looking at schedule J: http://www.irsx.info/#IRS990ScheduleJ
Then I went to the table details and picked the rows I wanted from the repeating group:
http://www.irsx.info/metadata/groups/SkdJRltdOrgOffcrTrstKyEmpl.html

Note that it's common for director/employee names in schedule J to get listed as businessname.

Also note that IRSx checks to see if a file has been downloaded before fetching it. Running this the first time will be slow if the filings aren't already downloaded, but much faster if they've already been downloaded.


In [14]:
DEMO_MAX = 10
num_rows = 0

for row in file_rows:
    num_rows += 1
    this_object_id = row['OBJECT_ID']
    parsed_filing = xml_runner.run_filing(this_object_id)
    
    # if it somehow busted, just note it and continue
    if not parsed_filing:
        print("Skipping filing %s(filings with pre-2013 schemas are skipped)\n row details: %s" % (this_object_id, row))
        continue 
    
    # Is there a Schedule J there at all?
    schedule_list = parsed_filing.list_schedules()
    if 'IRS990ScheduleJ' in schedule_list:
        
        # store the output in this dict
        outputdata = {}
        # assign some initial values from the input csv
        outputdata['period'] = row['TAX_PERIOD_x']
        outputdata['ein'] = row['EIN']
        outputdata['object_id'] = row['OBJECT_ID']
        outputdata['taxpayer_name'] = row['TAXPAYER_NAME']
        
        # some schedules can appear multiple times, but schedule j only appears once
        # so we grab the first one 
        parsed_skedj = parsed_filing.get_parsed_sked('IRS990ScheduleJ')[0] 
        # now use the table name we looked up -- that's where we find it
        try:
            # repeating groups are returned as an array of dicts
            employee_groups = parsed_skedj['groups']['SkdJRltdOrgOffcrTrstKyEmpl']
        except KeyError:
            print("No SkdJRltdOrgOffcrTrstKyEmpl found in %s skipping" % this_object_id)
            continue
          
        # read through each employee and pull out the data we want
        for employee_group in employee_groups:
            
            # That leaves the following values to come from schedule J if there is one
            # "name", "business_name1", "business_name2", "title", "org_comp", "related_comp"
            # those keys come from the headers we gave dictwriter before
            outputdata['name'] = employee_group.get('PrsnNm')
            outputdata['business_name1'] = employee_group.get('BsnssNmLn1Txt')
            outputdata['business_name2'] = employee_group.get('BsnssNmLn2Txt')
            outputdata['title'] = employee_group.get('TtlTxt')
            outputdata['org_comp'] = employee_group.get('TtlCmpnstnFlngOrgAmt') # Part II Column (E) 
            outputdata['related_comp'] = employee_group.get('TtlCmpnstnRltdOrgsAmt')
        
            dw.writerow(outputdata)
        
    else:
        print("No schedule J in filing %s, skipping" % this_object_id)

    # Don't run endlessly during a demo:
#    if(num_rows > DEMO_MAX):
#        break
    if num_rows%100==0:
        print("Processed %s filings" % num_rows)

No schedule J in filing 201802839349301235, skipping
No schedule J in filing 201832849349301958, skipping
No schedule J in filing 201840649349301254, skipping
No schedule J in filing 201732879349300213, skipping
No schedule J in filing 201832859349301623, skipping
No schedule J in filing 201811319349302571, skipping
No schedule J in filing 201802789349301415, skipping
No schedule J in filing 201823119349302177, skipping
No schedule J in filing 201713199349102316, skipping
No schedule J in filing 201713129349303356, skipping
No schedule J in filing 201842839349301304, skipping
No schedule J in filing 201842189349301159, skipping
No schedule J in filing 201802889349302065, skipping
No schedule J in filing 201802009349300520, skipping
No schedule J in filing 201723149349304102, skipping
No schedule J in filing 201812189349301451, skipping
No schedule J in filing 201812829349300536, skipping
No schedule J in filing 201832689349100608, skipping
Processed 100 filings
No schedule J in filing 

No schedule J in filing 201800579349300890, skipping
No schedule J in filing 201713199349302721, skipping
No schedule J in filing 201831909349301328, skipping
No schedule J in filing 201800649349300820, skipping
No schedule J in filing 201841359349313409, skipping
No schedule J in filing 201811839349300741, skipping
No schedule J in filing 201733219349301213, skipping
No schedule J in filing 201800269349300140, skipping
No schedule J in filing 201813139349302266, skipping
No schedule J in filing 201801209349301360, skipping
No schedule J in filing 201801359349102295, skipping
No schedule J in filing 201831359349313428, skipping
No schedule J in filing 201811849349300001, skipping
No schedule J in filing 201832159349100108, skipping
No schedule J in filing 201802889349301645, skipping
No schedule J in filing 201713189349309941, skipping
No schedule J in filing 201713189349311226, skipping
No schedule J in filing 201813129349302046, skipping
No schedule J in filing 201800589349300000, sk

No schedule J in filing 201821359349312377, skipping
No schedule J in filing 201841349349303244, skipping
No schedule J in filing 201820589349301027, skipping
No schedule J in filing 201801909349300630, skipping
No schedule J in filing 201821169349300327, skipping
No schedule J in filing 201811349349310616, skipping
No schedule J in filing 201840299349300344, skipping
No schedule J in filing 201831319349302433, skipping
No schedule J in filing 201713349349300111, skipping
No schedule J in filing 201801359349305000, skipping
No schedule J in filing 201802559349301185, skipping
No schedule J in filing 201823099349301842, skipping
No schedule J in filing 201723259349300022, skipping
No schedule J in filing 201713189349306236, skipping
No schedule J in filing 201820549349300112, skipping
No schedule J in filing 201822919349301397, skipping
No schedule J in filing 201811319349304471, skipping
No schedule J in filing 201840319349300729, skipping
No schedule J in filing 201802229349300330, sk

No schedule J in filing 201703459349100305, skipping
No schedule J in filing 201812749349100411, skipping
No schedule J in filing 201843059349100334, skipping
No schedule J in filing 201713399349300756, skipping
No schedule J in filing 201841359349103214, skipping
No schedule J in filing 201821979349300962, skipping
No schedule J in filing 201811359349305136, skipping
No schedule J in filing 201810879349301351, skipping
No schedule J in filing 201831949349100018, skipping
No SkdJRltdOrgOffcrTrstKyEmpl found in 201821939349300937 skipping
No schedule J in filing 201811299349302881, skipping
No schedule J in filing 201810239349301051, skipping
No schedule J in filing 201813169349101421, skipping
No schedule J in filing 201841139349300639, skipping
No schedule J in filing 201820309349301707, skipping
No schedule J in filing 201830309349300208, skipping
No schedule J in filing 201812279349301836, skipping
No schedule J in filing 201830459349301558, skipping
No schedule J in filing 20183043

No schedule J in filing 201713189349310566, skipping
No schedule J in filing 201833109349101103, skipping
No schedule J in filing 201723549349300942, skipping
No schedule J in filing 201811789349300336, skipping
No schedule J in filing 201841069349300809, skipping
No schedule J in filing 201822269349302522, skipping
No schedule J in filing 201821349349308377, skipping
No schedule J in filing 201703199349319655, skipping
No schedule J in filing 201820269349300247, skipping
No schedule J in filing 201842549349300724, skipping
No schedule J in filing 201743529349300904, skipping
No schedule J in filing 201801359349305350, skipping
No schedule J in filing 201831869349300953, skipping
No schedule J in filing 201810549349300321, skipping
No schedule J in filing 201733219349301058, skipping
No schedule J in filing 201831359349313793, skipping
No schedule J in filing 201830689349301143, skipping
No schedule J in filing 201812749349301301, skipping
No schedule J in filing 201743549349300724, sk

...

No schedule J in filing 201832159349200413, skipping
No schedule J in filing 201801349349201315, skipping
No schedule J in filing 201801279349101565, skipping
No schedule J in filing 201733199349207388, skipping
No schedule J in filing 201733199349205628, skipping
No schedule J in filing 201803099349201110, skipping
No schedule J in filing 201822959349200742, skipping
No schedule J in filing 201821309349201062, skipping
No schedule J in filing 201830369349200818, skipping
No schedule J in filing 201822779349300242, skipping
No schedule J in filing 201841779349200324, skipping
No schedule J in filing 201821359349102177, skipping
No schedule J in filing 201832079349100218, skipping
No schedule J in filing 201832719349100303, skipping
No schedule J in filing 201841439349200804, skipping
No schedule J in filing 201841359349102809, skipping
No schedule J in filing 201820189349200207, skipping
No schedule J in filing 201812929349200501, skipping
No schedule J in filing 201821599349200517, sk

No schedule J in filing 201703179349203085, skipping
No schedule J in filing 201743179349203249, skipping
No schedule J in filing 201801289349201625, skipping
No schedule J in filing 201810749349301631, skipping
No schedule J in filing 201831659349200533, skipping
No schedule J in filing 201831189349200103, skipping
No schedule J in filing 201842319349200004, skipping
No schedule J in filing 201841359349311794, skipping
No schedule J in filing 201832069349200728, skipping
No schedule J in filing 201703079349200100, skipping
No schedule J in filing 201733199349207943, skipping
No schedule J in filing 201713209349200506, skipping
No schedule J in filing 201813039349301401, skipping
No schedule J in filing 201840819349200149, skipping
No schedule J in filing 201723259349300737, skipping
No schedule J in filing 201822219349200817, skipping
No schedule J in filing 201832119349200733, skipping
No schedule J in filing 201800909349200315, skipping
No schedule J in filing 201811349349202926, sk

No schedule J in filing 201822329349200772, skipping
No schedule J in filing 201743199349201714, skipping
No schedule J in filing 201822859349201012, skipping
No schedule J in filing 201842839349200329, skipping
No schedule J in filing 201831239349100808, skipping
No schedule J in filing 201703189349200930, skipping
No schedule J in filing 201802349349200105, skipping
No schedule J in filing 201841349349200789, skipping
No schedule J in filing 201830959349200738, skipping
No schedule J in filing 201811249349200826, skipping
No schedule J in filing 201811349349204221, skipping
No schedule J in filing 201831359349102718, skipping
No schedule J in filing 201811359349101671, skipping
No schedule J in filing 201830529349200138, skipping
No schedule J in filing 201810609349200711, skipping
No schedule J in filing 201842499349200414, skipping
No schedule J in filing 201831349349205918, skipping
No schedule J in filing 201831569349200238, skipping
No schedule J in filing 201802919349200735, sk

No schedule J in filing 201810749349200776, skipping
No schedule J in filing 201832589349200108, skipping
No schedule J in filing 201832489349100213, skipping
No schedule J in filing 201733189349101078, skipping
No schedule J in filing 201713199349208611, skipping
No schedule J in filing 201833199349207158, skipping
No schedule J in filing 201743269349100504, skipping
No schedule J in filing 201810789349200856, skipping
No schedule J in filing 201801349349205520, skipping
No schedule J in filing 201821659349200327, skipping
No schedule J in filing 201822979349301382, skipping
No schedule J in filing 201832009349200768, skipping
No schedule J in filing 201743189349306344, skipping
No schedule J in filing 201832399349200818, skipping
No schedule J in filing 201820159349200802, skipping
No schedule J in filing 201843099349100339, skipping
No schedule J in filing 201840369349200764, skipping
No schedule J in filing 201812279349100336, skipping
No schedule J in filing 201703189349102545, sk

No schedule J in filing 201703119349100440, skipping
No schedule J in filing 201803109349100240, skipping
No schedule J in filing 201713189349102646, skipping
No schedule J in filing 201802889349100245, skipping
No schedule J in filing 201821629349100522, skipping
No schedule J in filing 201811209349100631, skipping
No schedule J in filing 201821419349100912, skipping
No schedule J in filing 201822609349100342, skipping
No schedule J in filing 201832339349100323, skipping
No schedule J in filing 201832539349100423, skipping
No schedule J in filing 201840189349300904, skipping
No schedule J in filing 201703189349308495, skipping
No schedule J in filing 201831649349300783, skipping
No schedule J in filing 201841309349202294, skipping
No schedule J in filing 201723109349300767, skipping
No schedule J in filing 201703169349200315, skipping
No schedule J in filing 201801349349205035, skipping
No schedule J in filing 201802999349200760, skipping
No schedule J in filing 201703139349201940, sk

In [15]:
# close the outfile
outfile.close()

sked_j_ore_efilers = pd.read_csv(outfilename)
sked_j_ore_efilers.head()

,period,ein,object_id,taxpayer_name,name,business_name1,business_name2,title,org_comp,related_comp
0,201612,941105628,201703139349302875,KAISER FOUNDATION HOSPITALS,Claudio F Abreu,NaN,NaN,"SVP, Regional IT Operations",0.0,1070022.0
1,201612,941105628,201703139349302875,KAISER FOUNDATION HOSPITALS,Gregory Adams,NaN,NaN,"EVP, Group President",0.0,3145675.0
2,201612,941105628,201703139349302875,KAISER FOUNDATION HOSPITALS,Ramon F Baez,NaN,NaN,Director,0.0,152591.0
3,201612,941105628,201703139349302875,KAISER FOUNDATION HOSPITALS,Mary Ann Barnes,NaN,NaN,Region President - Hawaii,0.0,1279544.0
4,201612,941105628,201703139349302875,KAISER FOUNDATION HOSPITALS,Anthony Barrueta,NaN,NaN,"SVP, Government Relations",0.0,1345769.0
